We are finding same product in both dataset based on similarity of both product.
we are going to take product with highest similarity score as match.
we are assumming that for each product match is allready there in another dataset.

Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import numpy as np
from numpy.linalg import norm
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.metrics.pairwise import cosine_similarity

Reading Files for 2 datasets

In [2]:
df_amz=pd.read_csv(r"C:\Users\suhas\Downloads\DS - Assignment Part 2 data set\amz_com-ecommerce_sample.csv",encoding="ISO-8859-1")
df_fl=pd.read_csv(r"C:\Users\suhas\Downloads\DS - Assignment Part 2 data set\flipkart_com-ecommerce_sample.csv")

Selecting particular columns for model

In [5]:
print("amazon:",df_amz.columns)
print("Flipkart:",df_fl.columns)

amazon: Index(['uniq_id', 'crawl_timestamp', 'product_url', 'product_name',
       'product_category_tree', 'pid', 'retail_price', 'discounted_price',
       'image', 'is_FK_Advantage_product', 'description', 'product_rating',
       'overall_rating', 'brand', 'product_specifications'],
      dtype='object')
Flipkart: Index(['uniq_id', 'crawl_timestamp', 'product_url', 'product_name',
       'product_category_tree', 'pid', 'retail_price', 'discounted_price',
       'image', 'is_FK_Advantage_product', 'description', 'product_rating',
       'overall_rating', 'brand', 'product_specifications'],
      dtype='object')


In [6]:
columns=["uniq_id","crawl_timestamp","product_url","image","product_rating","overall_rating"]
df_amz.drop(columns,axis=1,inplace=True)
df_fl.drop(columns,axis=1,inplace=True)

In [7]:
columns=['product_category_tree','description','brand','product_specifications']
for col in columns:
    df_amz[col]= df_amz[col].map(str)
for col in columns:
    df_fl[col]= df_fl[col].map(str)

In [12]:
def combined_features(row):
    return row['product_category_tree']+" "+row['pid']+" "+row['description']+" "+row['brand']+" "+row['product_specifications']
df_amz['combined features']=df_amz.apply(combined_features,axis=1)
df_fl['combined features']=df_fl.apply(combined_features,axis=1)

Creating similarity on combined features

In [14]:
cv=CountVectorizer ()
countmatrix=cv.fit_transform(df_amz['combined features'])
Data1=pd.DataFrame(countmatrix.toarray(),columns=cv.get_feature_names())
countmatrix1=cv.fit_transform(df_fl['combined features'])
Data2=pd.DataFrame(countmatrix1.toarray(),columns=cv.get_feature_names())

C:\Users\suhas\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [15]:
Columns=Data1.columns.intersection(Data2.columns)

In [ ]:
Data1=Data1[Columns]
Data2=Data2[Columns]

In [ ]:
data1=Data1.values
data2=Data2.values
df_sim=pd.DataFrame(columns=df_fl.index,index=df_amz.index)

Finding Similarity by mathematical formula of cosine similarity

In [19]:
k=0
l=0
for i in data1:
    for j in data2:
        cosine = np.dot(i,j)/(norm(i)*norm(j))  
        similarity.iloc[k,l]=cosine
        l+=1
    k+=1
    l=0

Based On similarity we will get most similar product in each dataset

In [8]:
df_sim=df_sim.astype('float')
#getting maximum value in each row
df=df_sim.idxmax(axis="columns")

In [14]:
col=["Product name in flipkart","Retail price in flipkart"
     ,"Discount proce in flipkart","Product name in amazon","Retail price in amazon","Discount proce in amazon"]
df_final=pd.DataFrame(columns=col)

In [18]:
for i,j in zip(df,df.index):
    amz_name=df_amz["product_name"].iloc[j]
    retail_amz=df_amz["retail_price"].iloc[j]
    discounted_amz=df_amz["discounted_price"].iloc[j]
    fl_name=df_fl["product_name"].iloc[i]
    retailfl=df_amz["retail_price"].iloc[i]
    discounted_fl=df_amz["discounted_price"].iloc[i]
    dict_df={"Product name in flipkart":fl_name,
         "Retail price in flipkart":retailfl,"Discount proce in flipkart":discounted_fl,
         "Product name in amazon":amz_name,
         "Retail price in amazon":retail_amz,"Discount proce in amazon":discounted_amz}
    df_final = df_final.append(dict_df, ignore_index = True)

In [17]:
df_final

,Product name in flipkart,Retail price in flipkart,Discount proce in flipkart,Product name in amazon,Retail price in amazon,Discount proce in amazon
0,FabHomeDecor Fabric Double Sofa Bed,32143,29121,FabHomeDecor Fabric Double Sofa Bed,32143,29121
1,FabHomeDecor Fabric Double Sofa Bed,32143,29121,FabHomeDecor Fabric Double Sofa Bed,32143,29121
2,AW Bellies,991,551,AW Bellies,991,551
3,Alisha Solid Women's Cycling Shorts,694,325,Alisha Solid Women's Cycling Shorts,694,325
4,Sicons All Purpose Arnica Dog Shampoo,208,258,Sicons All Purpose Arnica Dog Shampoo,208,258
